## Invoke SageMaker Enpoint from outside of AWS environment using SageMaker SDK

Model used: XGBoost Bike Rental Prediction Trained in the XGBoost Lectures  
  
This example uses the IAM user: ml_user_predict. The user was setup in the housekeeping lecture of the course.  

Refer to the lecture: Configure IAM Users, Setup Command Line Interface (CLI)

Ensure xgboost-biketrain-v1 Endpoint is deployed before running this example  
  
To create an endpoint using SageMaker Console:  
1. Select "Models" under "Inference" in navigation pane
2. Search for model using this prefix: xgboost-biketrain-v1
3. Select the latest model and choose create endpoint
4. Specify endpoint name as: xgboost-biketrain-v1
5. Create a new endpoint configuration
6. Create a new endpoint

In [1]:
# Install SageMaker 2.x version.
!pip install --upgrade sagemaker

  Created wheel for sagemaker: filename=sagemaker-2.16.3.post0-py2.py3-none-any.whl size=435632 sha256=b20b83741722ddbe2f60c0c7bbbe3628ee22a018e1fc42dc65f7414f74e70d78
  Stored in directory: C:\Users\ouraccount\AppData\Local\pip\Cache\wheels\69\e7\c0\e2be7e2bcfa81f7c2c192fc3d6cd32070ef02e38ac77e3056a
Successfully built sagemaker
  Found existing installation: smdebug-rulesconfig 0.1.4
    Uninstalling smdebug-rulesconfig-0.1.4:
      Successfully uninstalled smdebug-rulesconfig-0.1.4
  Found existing installation: sagemaker 1.72.1
    Uninstalling sagemaker-1.72.1:
      Successfully uninstalled sagemaker-1.72.1


In [4]:
import boto3
import sagemaker
import math
import dateutil

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [5]:
# Establish a session with AWS
# Specify credentials and region to be used for this session.
# We will use a ml_user_predict credentials that has limited privileges
boto_session = boto3.Session(profile_name='ml_user_predict',region_name='us-east-1')

In [6]:
sess = sagemaker.Session(boto_session=boto_session)

In [7]:
# Create a predictor and point to an existing endpoint

# Get Predictor using SageMaker SDK
# Specify Your Endpoint Name
endpoint_name = 'xgboost-biketrain-v1'

predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name,
                                                 sagemaker_session=sess)

In [8]:
# We are sending data for inference in CSV format
predictor.serializer = CSVSerializer()

In [9]:
#datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
# Actual=562
sample_one = '2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027'
# Actual=569
sample_two = '2012-12-19 18:00:00,4,0,1,1,15.58,19.695,50,23.9994'
# Actual=4
sample_three = '2012-12-10 01:00:00,4,0,1,2,14.76,18.94,100,0'

In [11]:
# Raw Data Structure: 
# datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count

# Model expects data in this format (it was trained with these features):
# season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour

def transform_data(data):
    features = data.split(',')
    
    # Extract year, month, day, dayofweek, hour
    dt = dateutil.parser.parse(features[0])

    features.append(str(dt.year))
    features.append(str(dt.month))
    features.append(str(dt.day))
    features.append(str(dt.weekday()))
    features.append(str(dt.hour))
    
    # Return the transformed data. skip datetime field
    return ','.join(features[1:])

In [12]:
print('Raw Data:\n',sample_one)
print('Transformed Data:\n',transform_data(sample_one))

Raw Data:
 2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027
Transformed Data:
 4,0,1,1,16.4,20.455,50,26.0027,2012,12,19,2,17


In [17]:
# Let's invoke prediction now
predictor.predict(transform_data(sample_one))

b'6.349300861358643'

In [18]:
# Actual Count is 562...but predicted is 6.3.

# Model was trained with log1p(count)
# So, we need to apply inverse transformation to get the actual count
# Predicted Count looks much better now
result = predictor.predict(transform_data(sample_one))
result = result.decode("utf-8")
print ('Predicted Count', math.expm1(float(result)))

Predicted Count 571.092597122044


In [24]:
# how to send multiple samples
result = predictor.predict([transform_data(sample_one), transform_data(sample_two)])

In [25]:
result.decode("utf-8")

'6.349300861358643,6.321451187133789'

In [26]:
# Batch Prediction
# Transform data and invoke prediction in specified batch sizes
def run_predictions(data, batch_size):
    predictions = []
    
    transformed_data = [transform_data(row.strip()) for row in data]
    
    for i in range(0, len(data), batch_size):
        
        print(i,i+batch_size)
        
        result = predictor.predict(transformed_data[i : i + batch_size])
        
        result = result.decode("utf-8")
        result = result.split(',')
        
        predictions += [math.expm1(float(r)) for r in result]
                
    return predictions

In [27]:
run_predictions([sample_one,sample_two,sample_three],10)

0 10


[571.092597122044, 555.3798181158465, 10.489585991183136]

In [28]:
# Run a batch prediction on Test.CSV File
# Read the file content
data = []
with open('test.csv','r') as f:
    # skip header
    f.readline()
    # Read remaining lines
    data = f.readlines()

In [29]:
len(data)

6493

In [30]:
%%time
predictions = run_predictions(data,500)

0 500
500 1000
1000 1500
1500 2000
2000 2500
2500 3000
3000 3500
3500 4000
4000 4500
4500 5000
5000 5500
5500 6000
6000 6500
Wall time: 1.21 s


In [31]:
len(predictions),len(data)

(6493, 6493)